In [216]:
import cv2
from skimage.filters import threshold_otsu
import numpy as np
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                               white_tophat)
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from skimage.draw import rectangle
from math import sqrt
%matplotlib inline 

In [240]:
im_in = cv2.imread("er1.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
###################Remove words###############
structureElm =  np.array((
	[1,1],
	[1,1]), dtype="int")
#closed = closing(im_th, structureElm)
#for i in range(5):
#im_th = closing(im_th, structureElm)
cv2.imwrite("closed.png", im_th)

#im2, contours_closed, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

True

In [241]:
# Mask used to flood filling.
# Notice the size needs to be 2 pixels than the image.
hImg, wImg = im_th.shape[:2]
area = hImg*wImg
mask = np.zeros((hImg+2, wImg+2), np.uint8)

im_floodfill = im_th.copy()
##################Flood fill corners#####
cv2.floodFill(im_floodfill, mask, (0,0), 255)
cv2.floodFill(im_floodfill, mask, (wImg-1,0), 255)
cv2.floodFill(im_floodfill, mask, (0,hImg-1), 255)
cv2.floodFill(im_floodfill, mask, (wImg-1,hImg-1), 255)

print("corners Filled")

corners Filled


In [242]:
######################cv contours#########
# Invert floodfilled image
im_floodfill_inv = cv2.bitwise_not(im_floodfill)
im2, contours_cv, hierarchy = cv2.findContours(im_floodfill_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
hierarchy = hierarchy[0]
#[Next, Previous, First_Child, Parent]
im_empty = np.ones((hImg, wImg,3), np.uint8) * 255
i=0
for cnt in contours_cv:
    #contours that has at least two children
    child1Idx = hierarchy[i][2]
    hasChildren = child1Idx != -1 and  hierarchy[child1Idx][0] != -1
    if(hasChildren):
        child2Idx = hierarchy[child1Idx][0]
    x,y,w,h = cv2.boundingRect(cnt)
    #this two children are close (letters)
    #check area
    if(hasChildren and h<=150 and w <=400):
        #print((w*h)/area)
        x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
        x2,y2,w2,h2 = cv2.boundingRect(contours_cv[child2Idx])
        if(sqrt((x1-x2)**2+((y1-y2)**2)) <= 100):
            cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
    if(child1Idx != -1 and  hierarchy[child1Idx][0] == -1):
        x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
        #print(w1/w)
        if(w1/w >= 0.3):
            cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
    i+=1

#cv2.drawContours(im_empty, contours_cv, -1, (0,0,0), 1)
#print(hierarchy)
cv2.imwrite("contoured.png", im_empty)
#########################################
# Display images.
cv2.imwrite("Thresholded_Image.png", im_th)
cv2.imwrite("Floodfilled_Image.png", im_floodfill)
cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)
im_th2 = ((255 - im_th)/255).astype(int)
im_ch = ((255 - im_empty[:,:,0])/255).astype(int)
anded = im_th2 & im_ch
cv2.imwrite("anded.png", anded*255)

#start flooding from corners?

0.5
0.5555555555555556
0.25
0.5
0.15384615384615385
0.25
0.25
0.2
0.18181818181818182
0.25
0.25
0.42857142857142855
0.25
0.5555555555555556
0.25
0.5
0.25
0.5
0.25
0.36363636363636365
0.5
0.25
0.42857142857142855
0.25
0.15384615384615385
0.5555555555555556
0.2
0.5


True

In [243]:
def isValid(imV,m, n, i, j):
    #print(i<0 , i>= m , j<0 , j>= n , imV[j][i] == 0)
    #print(i,j,m,n)
    if (i<0 or i>= m or j<0 or j>= n or imV[j][i] == 0):
        return False
    return True
 
# FloodFill function
def floodFillBFS(screen,m, n, xImg, yImg):
    queue = []
     
    # Append the position of starting
    # pixel of the component
    queue.append([xImg, yImg])
 
    # Color the pixel with the new color
    # if(screen[yImg][xImg]==0):
    #     #print("hi")
    #     return
    screen[yImg][xImg] = 0
 
    # While the queue is not empty i.e. the
    # whole component having prevC color
    # is not colored with newC color
    while queue:
        # Dequeue the front node
        currPixel = queue.pop() 
        posX = currPixel[0]
        posY = currPixel[1]
        # Check if the adjacent
        # pixels are valid
        if isValid(screen, m, n,posX + 1, posY):
           # print("valid r")
            screen[posY][posX + 1] = 0
            queue.append([posX + 1, posY])
         
        if isValid(screen, m, n,posX-1, posY):
           # print("valid l")
            screen[posY][posX-1]= 0
            queue.append([posX-1, posY])
         
        if isValid(screen, m, n, posX, posY + 1):
           # print("valid d")
            screen[posY + 1][posX]= 0
            queue.append([posX, posY + 1])
         
        if isValid(screen, m, n,posX, posY-1):
            #print("valid u")
            screen[posY-1][posX]= 0
            queue.append([posX, posY-1])

In [244]:
im_empty_2D=cv2.cvtColor(im_empty,cv2.COLOR_BGR2GRAY) 
for cnt in contours_closed:
    x,y,w,h = cv2.boundingRect(cnt)
    if w>20 and w<550 and h>5 and h<20:
        i = int((2*x+w)/2) 
        i = i if i>0 else 0
        j = y-1 if y>0 else 0
        im_empty[j][i] = [0,0,255]
        im_empty[y][x] = [0,255,0]
        im_empty[y][x+w] = [0,255,0]
        im_empty[y+h][x] = [0,255,0]
        im_empty[y+h][x+w] = [0,255,0]
        floodFillBFS(im_empty_2D,wImg,hImg, i,j)

cv2.imwrite("contoured.png", im_empty)


IndexError: index 810 is out of bounds for axis 0 with size 319

In [189]:
structureElm =  np.array((
	[1,1,1],
	[1,1,1],
    [1,1,1]), dtype="int")
im_empty_2D = dilation(im_empty_2D, structureElm)

cv2.imwrite("flooded.png", im_empty_2D)


True